#### Import Statements

In [1]:
import pandas as pd
import altair as alt
import numpy as np

import re

import time

import warnings

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import visibility_of_element_located

from difflib import SequenceMatcher



#### Import Scriptures from file

In [2]:
scriptures = pd.read_csv("C:/Users/LeRoy12358/Documents/School/W23/499 DS - Senior Project/WWPapers/raw data/lds-scriptures.csv")

# scriptures

#### Web Scraping

##### Scraping one page and extracting list of directions for other pages to scrape

In [3]:
browser = webdriver.Chrome()  # start a web browser
browser.get("https://scriptures.byu.edu/#:t3d092:p573")  # navigate to URL
# wait for page to load
# by waiting for <h1> element to appear on the page
title = (
    WebDriverWait(driver=browser, timeout=10)
    .until(visibility_of_element_located((By.ID, "jContent")))
    .text
)
# retrieve fully rendered HTML content
content = browser.page_source
browser.close()



In [4]:
# # we then could parse it with beautifulsoup
soup = BeautifulSoup(content, "html.parser")
# print(soup.find("jContent").text)
# soup

# soup.div['id']

# print(soup.prettify())
# content
# soup.div.div.div

soup.find("div", {"id": "jContent"})
tmp = soup.find('div', {'class': 'discourseBody'}).text

# tmp.replace("\xa0", "")
# tmp

In [5]:
links = soup.find('ul', {'class': 'talksblock'})

linkList = links.find_all('a', {'class': 'refcounter'})

link1 = links.find('a', {'class': 'refcounter'})['onclick']

# type(link1)
link1.replace("getTalk('", "").replace("');", "")
# for item in linkList:
#     # item.replace("getTalk('", "").replace("');", "")
#     # print(item)
#     pass
# linkList.replace("getTalk('", "").replace("');", "")

linkList2 = []

for link in soup.find_all('a', {'class': 'refcounter'}):
    linkList2.append(
        np.base_repr(int(
        link.get('onclick').replace("getTalk('", "").replace("');", "")
        ), 16
        ).lower()
        )
    
# http://example.com/elsie
# http://example.com/lacie
# http://example.com/tillie

# linkList2

##### Scraping all the pages

In [6]:
# This will likely take quite a while to run....

# breaker = False
browser = webdriver.Chrome()  # start a web browser
pot = []
rawPot = []

for i in range(0, len(linkList2)):
    url = linkList2[i]
    print(url, end = ", ")
    
    try:


        browser.get("https://scriptures.byu.edu/#:t" + url + ":p573")  # navigate to URL
        # wait for page to load by waiting for <h1> element to appear on the page
        title = (
            WebDriverWait(driver=browser, timeout=10)
            .until(visibility_of_element_located((By.ID, "jContent")))
            .text
        )
        # retrieve fully rendered HTML content
        content = browser.page_source

    except:
        print("Something broke, autofixing....", end = " ")
        browser.close()
        browser = webdriver.Chrome()
        i -= 1
        # continue
    else:
        # we then could parse it with beautifulsoup
        soup = BeautifulSoup(content, "html.parser")
        rawPot.append(soup.find('div', {'class': 'discourseBody'}))
        pot.append(soup.find('div', {'class': 'discourseBody'}).text)
        # pot.append(soup.find_all('a', {'href': 'discourseBody'}))
        time.sleep(.11)
    
browser.close()
    # if(breaker):
    #     break
    # breaker = True
    

3d092, 3a99c, Something broke, autofixing.... 3a987, 38296, 3827e, 38279, 35b8e, 35b8d, 35b8b, 35b75, 35b7e, 35b72, 35b79, 33473, 33471, Something broke, autofixing.... 3346f, 33466, 3345f, 2e662, Something broke, autofixing.... 2e65a, 2e648, Something broke, autofixing.... 2e653, 2bf3b, Something broke, autofixing.... 2bf36, 2bf2f, 2bf2e, 2bf25, 29834, Something broke, autofixing.... 2982e, 2981b, 27125, 27106, 24a1a, Something broke, autofixing.... 24a13, 249fc, 249f2, 222e1, Something broke, autofixing.... 222e6, 1fbe3, 1fbf5, 1d4f7, Something broke, autofixing.... 1d4c4, 1ade7, Something broke, autofixing.... 1add5, 1adba, 186cc, 186a3, Something broke, autofixing.... 15fd2, 15faf, Something broke, autofixing.... 15f9c, 138c6, 138c5, 1117f, ea75, Something broke, autofixing.... ea72, c37b, c363, Something broke, autofixing.... 9c7a, c35b, Something broke, autofixing.... 9c6c, Something broke, autofixing.... 9c66, 9c5f, Something broke, autofixing.... 9c54, 15fb9, Something broke, a

#### All the parsing

###### Extracting list of references from a single talk

In [122]:
# scriptures.verse_short_title = scriptures.verse_short_title.str.replace("JS—H", "JS-H")

In [123]:
# extracting a list of references 
def extractReferences(soup):
    citationList = soup.find_all('span', {'class': 'citation'})

    soupReferenceList = []
    for citation in citationList:
        soupReferenceList.append(
        citation.find_all('a', {'href': 'javascript:void(0)'})[1].text#.replace("—", "-")
        )
        # print(citation.find('span', {'class': 'citation'}))
    
    return soupReferenceList

# creating a variable containing only the first speech
tmp = rawPot[0]
referenceList = extractReferences(tmp)

print(referenceList)

['1 Ne. 13:12', 'D&C 135:3', 'D&C 136:39', 'Heb. 11:37', 'Rev. 6:9', '2 Pet. 1:21', 'Gen. 2:7', 'Isa. 31:1', 'Prov. 14:34', 'Gal. 6:7', 'Matt. 7:2', 'Isa. 40:1-3', 'Jer. 31:1-14', 'Ezek. 20:33-44', 'D&C 5:14', 'D&C 33:5', 'D&C 109:73', 'Eph. 2:20', '1 Pet. 2:6', 'Dan. 2:44', 'Dan. 2:34-35', 'D&C 65:2', 'Dan. 2:44', 'Isa. 2:2-3', 'Rev. 1:9', 'Rev. 14:6-7', 'Rev. 1:9', 'Rev. 6:9', 'D&C 13:1', 'D&C 27:8', 'JS—H 1:68-72', 'D&C 27:12', 'John 15:19', 'Isa. 60:22', 'Rom. 1:16', 'D&C 84:33', 'D&C 84:38-39', 'D&C 84:41', 'Micah 3:11', 'Acts 19:27', '1 Cor. 12:28', 'Eph. 2:20', 'Isa. 14:12', 'D&C 76:26', 'Jer. 25:31', 'Luke 21:24', 'Rev. 5:9', 'Rev. 8:6', 'Rev. 16:1', 'Rev. 21:9', 'Rev. 16:19', 'Isa. 34:5-6', 'D&C 1:36', 'Heb. 11:3', 'Heb. 11:7', 'Gen. 6:3', 'Moses 8:17', 'Heb. 11:8', 'Deut. 20:3', 'D&C 103:19', 'D&C 88:2', 'D&C 98:2', 'D&C 58:22', 'Hosea 8:7', '3 Ne. 27:27', 'Luke 17:33', 'Eph. 4:10', 'D&C 88:6', 'D&C 122:8', 'D&C 122:8', 'Rev. 2:10']


In [127]:
myRef = "JS—H 1:68"
scriptureFromReference(myRef)



JS—H 1:68


IndexError: single positional indexer is out-of-bounds

In [126]:
# This function takes a scripture reference as held in referenceList and return the scripture contents

def scriptureFromReference(reference):
    print(reference)
    if "-" in reference: # Isa. 40:1-3
        bookCh, verses = reference.split(":") #[Isa. 40, 1-3]
        if "-" in verses: #JS-H 1, 1-3 
            verseStart, verseEnd = verses.split("-") # Isa. 40, 1, 3
        else: # JS-H 1, 3
            verseStart = verseEnd = verses
        specificScripture = ""
        for i in range(int(verseStart), int(verseEnd) + 1):
            fullReference = bookCh + ":" + str(i)
            specificScripture = specificScripture + scriptures.query(
                "verse_short_title == @fullReference").scripture_text.iloc[0]
        return specificScripture
    else:
        specificScripture = scriptures.query("verse_short_title == @reference").scripture_text.iloc[0]
    return(specificScripture)

# result = scriptureFromReference(referenceList[3])#"Heb. 11:37")
# scriptureFromReference(referenceList[2])
# scriptureFromReference(referenceList[9])
# result

# takes an iterable list of scripture references and returns an iterable list of the contents of each scripture
def scriptureFromReferenceVectorized(refList):#, resultList):
    resultList = []
    # resultList = pd.Series(resultList, dtype = object)
    # print(type(resultList))
    for ref in refList:
        # resultList = pd.concat([resultList, pd.Series(scriptureFromReference(ref), dtype = object)])
        resultList.append(scriptureFromReference(ref))
    return resultList

# resultList = []
scripList = scriptureFromReferenceVectorized(referenceList)
# scriptureFromReferenceVectorized([referenceList[0], referenceList[1], referenceList[2]])
# type(referenceList)
# referenceList.str.len()

# scriptureFromReference("Isa. 40:1-3")
# scripList

1 Ne. 13:12
D&C 135:3
D&C 136:39
Heb. 11:37
Rev. 6:9
2 Pet. 1:21
Gen. 2:7
Isa. 31:1
Prov. 14:34
Gal. 6:7
Matt. 7:2
Isa. 40:1-3
Jer. 31:1-14
Ezek. 20:33-44
D&C 5:14
D&C 33:5
D&C 109:73
Eph. 2:20
1 Pet. 2:6
Dan. 2:44
Dan. 2:34-35
D&C 65:2
Dan. 2:44
Isa. 2:2-3
Rev. 1:9
Rev. 14:6-7
Rev. 1:9
Rev. 6:9
D&C 13:1
D&C 27:8
JS—H 1:68-72


IndexError: single positional indexer is out-of-bounds

In [ ]:
# JS—H 1:68-72
# "—" in "-"
# "-" in "—"

False

In [ ]:
# resultList = []
# # resultList = pd.Series(resultList, dtype = object)
# # print(type(resultList))
# for ref in referenceList:
#     # resultList = pd.concat([resultList, pd.Series(scriptureFromReference(ref), dtype = object)])
#     resultList.append(scriptureFromReference(ref))
# # print(scriptureFromReference(ref))
# # print(scriptureFromReferenceVectorized(referenceList))
# print(resultList)

In [ ]:
myTmp = pd.Series(rawPot[0].text.replace("\xa0", "").replace("—", "-"))

surrCharSets = pd.DataFrame(columns = ["charSet", "ref"])
# surrCharSets.columns = ["charSet", "ref"]

for ref in referenceList:
    # this bit to suppress warnings about regex boolean values
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        singleCharPair = myTmp.str.extract("(.{100})(" + ref + ")")
        singleCharPair.columns = ["charSet", "ref"]
        surrCharSets = pd.concat([surrCharSets, singleCharPair], ignore_index= True)
        # remove the just found instance of the reference and only that instance (n = 1)
        myTmp = myTmp.str.replace(ref, "", n = 1)
    # print(ref, end = "")

In [ ]:
type(referenceList)
# type(list(surrCharSets['ref']))

list

In [ ]:
resultList2 = []
# resultList2 = pd.Series(resultList2)

# resultList2 = 
# scriptureFromReferenceVectorized(referenceList, resultList2)

# print(resultList2)

# surrCharSets['scriptureContents'] = resultList2
surrCharSets['scriptureContents'] = scriptureFromReferenceVectorized(list(surrCharSets['ref']))
# surrCharSets['scriptureContents'] = scriptureFromReferenceVectorized(referenceList)

print(referenceList)
list(surrCharSets.ref)




['1 Ne. 13:12', 'D&C 135:3', 'D&C 136:39', 'Heb. 11:37', 'Rev. 6:9', '2 Pet. 1:21', 'Gen. 2:7', 'Isa. 31:1', 'Prov. 14:34', 'Gal. 6:7', 'Matt. 7:2', 'Isa. 40:1-3', 'Jer. 31:1-14', 'Ezek. 20:33-44', 'D&C 5:14', 'D&C 33:5', 'D&C 109:73', 'Eph. 2:20', '1 Pet. 2:6', 'Dan. 2:44', 'Dan. 2:34-35', 'D&C 65:2', 'Dan. 2:44', 'Isa. 2:2-3', 'Rev. 1:9', 'Rev. 14:6-7', 'Rev. 1:9', 'Rev. 6:9', 'D&C 13:1', 'D&C 27:8', 'JS-H 1:68-72', 'D&C 27:12', 'John 15:19', 'Isa. 60:22', 'Rom. 1:16', 'D&C 84:33', 'D&C 84:38-39', 'D&C 84:41', 'Micah 3:11', 'Acts 19:27', '1 Cor. 12:28', 'Eph. 2:20', 'Isa. 14:12', 'D&C 76:26', 'Jer. 25:31', 'Luke 21:24', 'Rev. 5:9', 'Rev. 8:6', 'Rev. 16:1', 'Rev. 21:9', 'Rev. 16:19', 'Isa. 34:5-6', 'D&C 1:36', 'Heb. 11:3', 'Heb. 11:7', 'Gen. 6:3', 'Moses 8:17', 'Heb. 11:8', 'Deut. 20:3', 'D&C 103:19', 'D&C 88:2', 'D&C 98:2', 'D&C 58:22', 'Hosea 8:7', '3 Ne. 27:27', 'Luke 17:33', 'Eph. 4:10', 'D&C 88:6', 'D&C 122:8', 'D&C 122:8', 'Rev. 2:10']


['1 Ne. 13:12',
 'D&C 135:3',
 'D&C 136:39',
 'Heb. 11:37',
 'Rev. 6:9',
 '2 Pet. 1:21',
 'Gen. 2:7',
 'Isa. 31:1',
 'Prov. 14:34',
 'Gal. 6:7',
 'Matt. 7:2',
 'Isa. 40:1-3',
 'Jer. 31:1-14',
 'Ezek. 20:33-44',
 'D&C 5:14',
 'D&C 33:5',
 'D&C 109:73',
 'Eph. 2:20',
 '1 Pet. 2:6',
 'Dan. 2:44',
 'Dan. 2:34-35',
 'D&C 65:2',
 'Dan. 2:44',
 'Isa. 2:2-3',
 'Rev. 1:9',
 'Rev. 14:6-7',
 'Rev. 1:9',
 'Rev. 6:9',
 'D&C 13:1',
 'D&C 27:8',
 'JS-H 1:68-72',
 'D&C 27:12',
 'John 15:19',
 'Isa. 60:22',
 'Rom. 1:16',
 'D&C 84:33',
 'D&C 84:38-39',
 'D&C 84:41',
 'Micah 3:11',
 'Acts 19:27',
 '1 Cor. 12:28',
 'Eph. 2:20',
 'Isa. 14:12',
 'D&C 76:26',
 'Jer. 25:31',
 'Luke 21:24',
 'Rev. 5:9',
 'Rev. 8:6',
 'Rev. 16:1',
 'Rev. 21:9',
 'Rev. 16:19',
 'Isa. 34:5-6',
 'D&C 1:36',
 'Heb. 11:3',
 'Heb. 11:7',
 'Gen. 6:3',
 'Moses 8:17',
 'Heb. 11:8',
 'Deut. 20:3',
 'D&C 103:19',
 'D&C 88:2',
 'D&C 98:2',
 'D&C 58:22',
 'Hosea 8:7',
 '3 Ne. 27:27',
 'Luke 17:33',
 'Eph. 4:10',
 'D&C 88:6',
 'D&C 122:8',
 

In [ ]:
# for ref in surrCharSets.ref:
#     print(ref)

In [ ]:
"-" in 'JS—H 1:'

False

In [ ]:
pd.set_option('display.max_rows', 300)
# list(surrCharSets.ref)
surrCharSets

,charSet,ref,scriptureContents
0,ases. When Columbus was moved upon by the Spir...,1 Ne. 13:12,And I looked and beheld a man among the Gentil...
1,elator-who escaped. They all died violent deat...,D&C 135:3,"Joseph Smith, the Prophet and Seer of the Lord..."
2,elator-who escaped. They all died violent deat...,D&C 136:39,Many have marveled because of his death; but i...
3,s. They had to seal their testimony with their...,Heb. 11:37,"They were stoned, they were sawn asunder, were..."
4,"under, beheaded, or in some way put to death f...",Rev. 6:9,"And when he had opened the fifth seal, I saw u..."
5,record through the medium of holy men who wro...,2 Pet. 1:21,For the prophecy came not in old time by the w...
6,destiny of the whole world-of the fourteen hun...,Gen. 2:7,And the LORD God formed man of the dust of the...
7,use the world is not governed by the spirit of...,Isa. 31:1,WOE to them that go down to Egypt for help; an...
8,"later, has manifested itself: that virtue exa...",Prov. 14:34,Righteousness exalteth a nation: but sin is a ...
9,y under heaven from the foundation of the worl...,Gal. 6:7,Be not deceived; God is not mocked: for whatso...


In [ ]:
print(redundantReferenceList)
# print(list(surrCharSets.ref))
print(referenceList)

['1 Ne. 13:12', 'D&C 135:3', 'D&C 136:39', 'Heb. 11:37', 'Rev. 6:9', '2 Pet. 1:21', 'Gen. 2:7', 'Isa. 31:1', 'Prov. 14:34', 'Gal. 6:7', 'Matt. 7:2', 'Isa. 40:1-3', 'Jer. 31:1-14', 'Ezek. 20:33-44', 'D&C 5:14', 'D&C 33:5', 'D&C 109:73', 'Eph. 2:20', '1 Pet. 2:6', 'Dan. 2:44', 'Dan. 2:34-35', 'D&C 65:2', 'Dan. 2:44', 'Isa. 2:2-3', 'Rev. 1:9', 'Rev. 14:6-7', 'Rev. 1:9', 'Rev. 6:9', 'D&C 13:1', 'D&C 27:8', 'JS-H 1:68-72', 'D&C 27:12', 'John 15:19', 'Isa. 60:22', 'Rom. 1:16', 'D&C 84:33', 'D&C 84:38-39', 'D&C 84:41', 'Micah 3:11', 'Acts 19:27', '1 Cor. 12:28', 'Eph. 2:20', 'Isa. 14:12', 'D&C 76:26', 'Jer. 25:31', 'Luke 21:24', 'Rev. 5:9', 'Rev. 8:6', 'Rev. 16:1', 'Rev. 21:9', 'Rev. 16:19', 'Isa. 34:5-6', 'D&C 1:36', 'Heb. 11:3', 'Heb. 11:7', 'Gen. 6:3', 'Moses 8:17', 'Heb. 11:8', 'Deut. 20:3', 'D&C 103:19', 'D&C 88:2', 'D&C 98:2', 'D&C 58:22', 'Hosea 8:7', '3 Ne. 27:27', 'Luke 17:33', 'Eph. 4:10', 'D&C 88:6', 'D&C 122:8', 'D&C 122:8', 'Rev. 2:10']
['1 Ne. 13:12', 'D&C 135:3', 'D&C 136:39', 

In [ ]:
['1 Ne. 13:12', 'D&C 135:3', 'D&C 136:39', 'Heb. 11:37', 'Rev. 6:9', '2 Pet. 1:21', 'Gen. 2:7', 'Isa. 31:1', 'Prov. 14:34', 'Gal. 6:7', 'Matt. 7:2', 'Isa. 40:1-3', 'Jer. 31:1-14', 'Ezek. 20:33-44', 'D&C 5:14', 'D&C 33:5', 'D&C 109:73', 'Eph. 2:20', '1 Pet. 2:6', 'Dan. 2:44', 'Dan. 2:34-35', 'D&C 65:2', 'Dan. 2:44', 'Isa. 2:2-3', 'Rev. 1:9', 'Rev. 14:6-7', 'Rev. 1:9', 'Rev. 6:9', 'D&C 13:1', 'D&C 27:8', 
nan,
 'D&C 27:12', 'John 15:19', 'Isa. 60:22', 'Rom. 1:16', 'D&C 84:33', 'D&C 84:38-39', 'D&C 84:41', 'Micah 3:11', 'Acts 19:27', '1 Cor. 12:28', 'Eph. 2:20', 'Isa. 14:12', 'D&C 76:26', 'Jer. 25:31', 'Luke 21:24', 'Rev. 5:9', 'Rev. 8:6', 'Rev. 16:1', 'Rev. 21:9', 'Rev. 16:19', 'Isa. 34:5-6', 'D&C 1:36', 'Heb. 11:3', 'Heb. 11:7', 'Gen. 6:3', 'Moses 8:17', 'Heb. 11:8', 'Deut. 20:3', 'D&C 103:19', 'D&C 88:2', 'D&C 98:2', 'D&C 58:22', 'Hosea 8:7', '3 Ne. 27:27', 'Luke 17:33', 'Eph. 4:10', 'D&C 88:6', 'D&C 122:8', 'D&C 122:8', 'Rev. 2:10']
['1 Ne. 13:12', 'D&C 135:3', 'D&C 136:39', 'Heb. 11:37', 'Rev. 6:9', '2 Pet. 1:21', 'Gen. 2:7', 'Isa. 31:1', 'Prov. 14:34', 'Gal. 6:7', 'Matt. 7:2', 'Isa. 40:1-3', 'Jer. 31:1-14', 'Ezek. 20:33-44', 'D&C 5:14', 'D&C 33:5', 'D&C 109:73', 'Eph. 2:20', '1 Pet. 2:6', 'Dan. 2:44', 'Dan. 2:34-35', 'D&C 65:2', 'Dan. 2:44', 'Isa. 2:2-3', 'Rev. 1:9', 'Rev. 14:6-7', 'Rev. 1:9', 'Rev. 6:9', 'D&C 13:1', 'D&C 27:8', 
'JS-H 1:68-72',
 'D&C 27:12', 'John 15:19', 'Isa. 60:22', 'Rom. 1:16', 'D&C 84:33', 'D&C 84:38-39', 'D&C 84:41', 'Micah 3:11', 'Acts 19:27', '1 Cor. 12:28', 'Eph. 2:20', 'Isa. 14:12', 'D&C 76:26', 'Jer. 25:31', 'Luke 21:24', 'Rev. 5:9', 'Rev. 8:6', 'Rev. 16:1', 'Rev. 21:9', 'Rev. 16:19', 'Isa. 34:5-6', 'D&C 1:36', 'Heb. 11:3', 'Heb. 11:7', 'Gen. 6:3', 'Moses 8:17', 'Heb. 11:8', 'Deut. 20:3', 'D&C 103:19', 'D&C 88:2', 'D&C 98:2', 'D&C 58:22', 'Hosea 8:7', '3 Ne. 27:27', 'Luke 17:33', 'Eph. 4:10', 'D&C 88:6', 'D&C 122:8', 'D&C 122:8', 'Rev. 2:10']


NameError: name 'nan' is not defined

In [ ]:
# # surrCharSets['refContents'] = scriptureFromReferenceVectorized(surrCharSets.ref)
# # surrCharSets['refContents'] = scriptureFromReferenceVectorized(surrCharSets.ref)
# # surrCharSets = surrCharSets.assign(refContents = lambda x: scriptureFromReference(x.ref))

# # map(fun, iter)

list(
    map(
    scriptureFromReference, list(surrCharSets.ref)
    )
    )

# redundantReferenceList = []
# for item in surrCharSets.ref:
#     with warnings.catch_warnings():
#         warnings.simplefilter("ignore")
#         redundantReferenceList.append(item)

# list(
#     map(
#     scriptureFromReference, redundantReferenceList#list(surrCharSets.ref)
#     )
#     )
# # surrCharSets['refContents'] = 3

# # surrCharSets

['And I looked and beheld a man among the Gentiles, who was separated from the seed of my brethren by the many waters; and I beheld the Spirit of God, that it came down and wrought upon the man; and he went forth upon the many waters, even unto the seed of my brethren, who were in the promised land.',
 "Joseph Smith, the Prophet and Seer of the Lord, has done more, save Jesus only, for the salvation of men in this world, than any other man that ever lived in it. In the short space of twenty years, he has brought forth the Book of Mormon, which he translated by the gift and power of God, and has been the means of publishing it on two continents; has sent the fulness of the everlasting gospel, which it contained, to the four quarters of the earth; has brought forth the revelations and commandments which compose this book of Doctrine and Covenants, and many other wise documents and instructions for the benefit of the children of men; gathered many thousands of the Latter-day Saints, found

In [ ]:
singleRefCharPairSeries = myTmp.str.extract("(.{100})(" + ref + ")")
singleRefCharPairDf = pd.DataFrame(data = {"charSet": singleRefCharPairSeries[0], "ref": singleRefCharPairSeries[1]})
# type(singleRefCharPairDf)
singleRefCharPairSeries.columns = ["charSet", "ref"]
pd.concat([singleRefCharPairSeries, singleRefCharPairDf])
# singleRefCharPairDf

surrCharSets

,charSet,ref,scriptureContents
0,ases. When Columbus was moved upon by the Spir...,1 Ne. 13:12,And I looked and beheld a man among the Gentil...
1,elator—who escaped. They all died violent deat...,D&C 135:3,"Joseph Smith, the Prophet and Seer of the Lord..."
2,elator—who escaped. They all died violent deat...,D&C 136:39,Many have marveled because of his death; but i...
3,s. They had to seal their testimony with their...,Heb. 11:37,"They were stoned, they were sawn asunder, were..."
4,"under, beheaded, or in some way put to death f...",Rev. 6:9,"And when he had opened the fifth seal, I saw u..."
...,...,...,...
66,since the creation of the world for the sake o...,Eph. 4:10,He that descended is the same also that ascend...
67,since the creation of the world for the sake o...,D&C 88:6,"He that ascended up on high, as also he descen..."
68,since the creation of the world for the sake o...,D&C 122:8,The Son of Man hath descended below them all. ...
69,"descended, I believe, lower than any other man...",D&C 122:8,The Son of Man hath descended below them all. ...


In [ ]:
pd.set_option('display.max_rows', 100)

# s[0]

In [ ]:
pd.DataFrame(s.str.extractall(
    "(.{0,30})(\d* *\w*&?\w*\.? *\d+:\d+-?\d*)"
    )).reset_index().drop(columns=[])


,level_0,match,0,1
0,0,0,"ean to find a new continent,",1 Ne. 13:12
1,0,1,testimony with their blood.,D&C 135:3
2,0,2,D&C 13,6:39
3,0,3,"d; others were sawn asunder,",Heb. 11:37
4,0,4,e testimony of Jesus Christ.,Rev. 6:9
...,...,...,...,...
2207,46,64,showing that summer is nigh.,Matt. 24:32
2208,46,65,"e among the foolish virgins,",Matt. 25:1-13
2209,46,66,D&C 6,3:54
2210,46,67,"tand the signs of the times,",Matt. 16:3


In [ ]:

def findMatches(str1, str2, BLOCK_MIN_LENGTH = 8):
    sm = SequenceMatcher(lambda x: x in " ", str1, str2)
    finalMatches = []
    matchesPosns = sm.get_matching_blocks()

    # put together a list of matches longer than only a couple characters
    for matchPos in matchesPosns:
        matchLength = matchPos[2]
        if matchLength < BLOCK_MIN_LENGTH:
            continue
        # print(str1[matchPos[0]:matchPos[0] + matchPos[2]])
        finalMatches.append(matchPos)
    return [str1[finalMatches[0][0]:finalMatches[-1][0] + finalMatches[-1][2]],
            str2[finalMatches[0][1]:finalMatches[-1][1] + finalMatches[-1][2]]]


In [ ]:
# TODO: need more training data / specific checks for this findMatches function

myStr1 = "He has left on record through the medium of holy men who wrote and spoke as they were moved upon by the Holy Ghost"
myStr2 = "For the prophecy came not in old time by the will of man: but holy men of God spake as they were moved by the Holy Ghost."

findMatches(myStr1, myStr2)

[' holy men who wrote and spoke as they were moved upon by the Holy Ghost',
 ' holy men of God spake as they were moved by the Holy Ghost']

In [ ]:
for row in 

##### String Matching Attempt using vectorization of words
Computation-intensive

In [ ]:

# pip install -U sentence-transformers

from sentence_transformers import SentenceTransformer, util


In [ ]:

sentences = ["I'm happy", "I'm full of happiness"]


In [ ]:

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


In [ ]:

#Compute embedding for both lists
# Note from E: interesting - encodings must be computed individually....
embedding_1= model.encode(sentences[0], convert_to_tensor=True) 
embedding_2 = model.encode(sentences[1], convert_to_tensor=True)


In [ ]:

util.pytorch_cos_sim(embedding_1, embedding_2)
## tensor([[0.6003]])



tensor([[0.6003]])

In [ ]:

## TODO for the upgraded string matching function:
# convert each string into a list of individual words
# --- ACTUALLY DON'T - because the encoding functions are for the full strings
# compute similarity score for the sentence pair (one score - to each other)
# save score and both 'current best phrases' (perhaps remember top 5 values and pairs?)
# repetitively cut off words from the start/end of first/second sentence 
# --- (alternating? - NOT ALTERNATING - NEED A CLEAR SIGNAL OF "DEAD END")
# track recent sim scores and when sim score drops for too many iterations in a row, revert to "best"
# --- plus a couple words? but def less than previous 'max'?
# --- comparing how fast match rate rises/drops?